In [1]:
from Bio import Entrez
from xml.etree import ElementTree as ET
import pandas as pd
import time

# Set email for NCBI Entrez
Entrez.email = "ainaabadilazaro@gmail.com"

# Define your search term and number of articles
search_term = "intestinal organoids"
max_results = 1000

# Step 1: Search PubMed for the query
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=max_results, sort="relevance")
record = Entrez.read(handle)
handle.close()
pmids = record["IdList"]

# Step 2: Fetch the article details
def fetch_details(id_list):
    ids = ",".join(id_list)
    handle = Entrez.efetch(db="pubmed", id=ids, rettype="abstract", retmode="xml")
    records = ET.parse(handle).getroot()
    return records

records = fetch_details(pmids)

# Step 3: Parse titles, abstracts, and publication dates
results = []
for article in records.findall(".//PubmedArticle"):
    try:
        pmid = article.findtext(".//PMID")
        title = article.findtext(".//ArticleTitle")
        abstract = article.findtext(".//Abstract/AbstractText")
        
        # Try to extract year from PubDate
        pub_date = article.find(".//PubDate")
        if pub_date is not None:
            year = pub_date.findtext("Year")
            medline_date = pub_date.findtext("MedlineDate")
            if year:
                pub_year = year
            elif medline_date:
                pub_year = medline_date.split(" ")[0]
            else:
                pub_year = None
        else:
            pub_year = None
        
        if abstract:  # Skip articles with no abstract
            results.append({
                "pmid": pmid,
                "title": title,
                "abstract": abstract,
                "year": pub_year
            })
    except Exception as e:
        print("Error parsing article:", e)
        continue
    time.sleep(0.1)  # Respect NCBI rate limits

# Step 4: Save as DataFrame
df = pd.DataFrame(results)
print(df.head())

# Step 5: Save the DataFrame to a CSV file
df.to_csv("intestinal_organoids_articles.csv", index=False)

       pmid                                              title  \
0  32940424  Establishment and Culture of Human Intestinal ...   
1  35110738  Transplantation of intestinal organoids into a...   
2  30526881  Human Intestinal Organoids Maintain Self-Renew...   
3  38816841  Current applications of intestinal organoids: ...   
4  31216420  Gut organoids: mini-tissues in culture to stud...   

                                            abstract  year  
0  Human intestinal organoids derived from adult ...  2020  
1  Intestinal organoids are fundamental in vitro ...  2022  
2  Cellular diversity that shapes tissue architec...  2018  
3  In the past decade, intestinal organoid techno...  2024  
4  In vitro, cell cultures are essential tools in...  2019  
